In [1]:
import tensorflow as tf
import tensorflow_ranking as tfr
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

In [2]:
tf.compat.v1.disable_eager_execution()

In [3]:
class IteratorInitializerHook(tf.estimator.SessionRunHook):
    """initialize the iterator"""
    
    def __init__(self):
        super(IteratorInitializerHook, self).__init__()
        self.iterator_initializer_fn = None
        
    def after_create_session(self, session, coord):
        """Initialize the iterator after the session has been created"""
        del coord
        self.iterator_initializer_fn(session)

In [4]:
def example_feature_columns():
    """return the example feature columns"""
    return {
        name:
        tf.feature_column.numeric_column(name, default_value=0.0, shape=(1,))
        for name in feature_names
    }

In [5]:
def dataframe_transformer_train(df):
    
    """
    transform dataframe into feature_map dict and labels list
    the shape of each feature is [query_num, list_size, 1]
    the shape of labels list is [query_num, list_size]
    feature_map and labels will be convert to tensorflow dataset later
     
    """
    queryIDs = df['#QueryID'].unique() # get the list of unique Query IDs
    df_grouped = df.groupby('#QueryID') # group the dataset by query id 
    
    feature_map = {name:[] for name in feature_names}
    label_list = []
        
    for qid in queryIDs:
            
        df_tmp = df_grouped.get_group(qid)
            
        feature_map_tmp = {name:np.array(df_tmp[name]) for name in feature_names}
        feature_map_tmp = {
            name:
            np.pad(value, (0,list_size-len(value)), 'constant', constant_values=0) 
            for name, value in feature_map_tmp.items()
        }
        feature_map_tmp = {
            name:
            np.reshape(value, (list_size,1))
            for name, value in feature_map_tmp.items()
        }
        for key in feature_map:
            feature_map[key].append(feature_map_tmp[key])
            
        label_list_tmp = np.array(df_tmp['Label'])
        label_list_tmp = np.pad(label_list_tmp, (0, list_size-len(label_list_tmp)), 'constant', constant_values=-1.)
        label_list.append(label_list_tmp)
            
    feature_map = {name:np.array(value).astype('float32') for name, value in feature_map.items()}  # convert the values of the dict to np array
        
    return feature_map, np.array(label_list).astype('float32')

In [6]:
def dataframe_transformer_pred(df):
    
    queryIDs = df['#QueryID'].unique()
    df_grouped = df.groupby('#QueryID')   
    qid_to_docs = {qid:df_grouped.get_group(qid)['Docid'].values for qid in queryIDs}
   
    feature_map = {name:[] for name in feature_names}
    
    for qid in queryIDs:
        df_tmp = df_grouped.get_group(qid)
        feature_map_tmp = {name:np.array(df_tmp[name]) for name in feature_names}
        feature_map_tmp = {
            name:
            np.pad(value, (0,list_size-len(value)), 'constant', constant_values=0) 
            for name, value in feature_map_tmp.items()
        }
        feature_map_tmp = {
            name:
            np.reshape(value, (list_size,1))
            for name, value in feature_map_tmp.items()
        }
        for key in feature_map:
            feature_map[key].append(feature_map_tmp[key])
            
    feature_map = {name:np.array(value).astype('float32') for name, value in feature_map.items()}
    return feature_map, qid_to_docs

In [7]:
def get_train_inputs(features, labels, batch_size):
    """set up training input function that meets tensorflow's requirements"""
    iterator_initializer_hook = IteratorInitializerHook()
    
    def _train_input_fn():
        """define training input function"""
    
        # create feature placeholder
        feature_placeholder = {
            key:tf.compat.v1.placeholder(value.dtype, value.shape)
            for key,value in features.items()
        }
    
        # create label placeholder
        label_placeholder = tf.compat.v1.placeholder(labels.dtype, labels.shape)
    
        # create dataset that meet tensorflow's requirement
        dataset = tf.data.Dataset.from_tensor_slices((feature_placeholder, label_placeholder))
        dataset = dataset.shuffle(1000).repeat().batch(batch_size)
        iterator = tf.compat.v1.data.make_initializable_iterator(dataset)
        
        feed_dict = {label_placeholder:labels}
        feed_dict.update(
            {feature_placeholder[key]:features[key] for key in feature_placeholder})
        iterator_initializer_hook.iterator_initializer_fn = (
            lambda sess: sess.run(iterator.initializer, feed_dict=feed_dict))
        return iterator.get_next()
    
    return _train_input_fn, iterator_initializer_hook

In [8]:
def get_vali_inputs(features, labels):
    """set up validation inputs in a single batch"""
    iterator_initializer_hook = IteratorInitializerHook()
    
    def _vali_input_fn():
        """define validation input function"""
        feature_placeholder = {
            key:tf.compat.v1.placeholder(value.dtype, value.shape)
            for key,value in features.items()
        }
        label_placeholder = tf.compat.v1.placeholder(labels.dtype, labels.shape)
        dataset = tf.data.Dataset.from_tensors((feature_placeholder, label_placeholder))
        iterator = tf.compat.v1.data.make_initializable_iterator(dataset)
        feed_dict = {label_placeholder:labels}
        feed_dict.update(
            {feature_placeholder[key]:features[key] for key in feature_placeholder})
        iterator_initializer_hook.iterator_initializer_fn = (
            lambda sess: sess.run(iterator.initializer, feed_dict=feed_dict))
        return iterator.get_next()
    
    return _vali_input_fn, iterator_initializer_hook

In [9]:
def get_predict_inputs(features):
    
    iterator_initializer_hook = IteratorInitializerHook()
    
    def _predict_input_fn():
        
        feature_placeholder = {
            key:tf.compat.v1.placeholder(value.dtype, value.shape)
            for key,value in features.items()
        }
        dataset = tf.data.Dataset.from_tensors(feature_placeholder)
        iterator = tf.compat.v1.data.make_initializable_iterator(dataset)
        feed_dict = {}
        feed_dict.update(
            {feature_placeholder[key]:features[key] for key in feature_placeholder})
        iterator_initializer_hook.iterator_initializer_fn = (
            lambda sess: sess.run(iterator.initializer, feed_dict=feed_dict))
        return iterator.get_next()
    
    return _predict_input_fn, iterator_initializer_hook

In [10]:
def make_serving_input_fn():
    """return srving input fn to receive tf.Example"""
    feature_spec = tf.feature_column.make_parse_example_spec(example_feature_columns().values())
    return tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)

In [11]:
def make_transform_fn():
    """returns a transform fuction that converts features to dense tensors"""
    
    def _transform_fn(features, mode):
        """define transform_fn"""
        context_features, example_features = (
            tfr.feature.encode_listwise_features(
                features=features, 
                context_feature_columns=None, 
                example_feature_columns=example_feature_columns(), 
                mode=mode, 
                scope='transform_layer'))
            
        return context_features, example_features
    
    return _transform_fn

In [12]:
def make_score_fn(hparams):
    """return a score function"""
    
    def _score_fn(context_features, group_features, mode, params, config):
        """define the network to score documents"""
        with tf.compat.v1.name_scope('input_layer'):
            group_input = [
                tf.compat.v1.layers.flatten(group_features[name])
                for name in sorted(example_feature_columns())
            ]
            input_layer = tf.concat(group_input, 1)
            tf.compat.v1.summary.scalar('input_sparsity', tf.nn.zero_fraction(input_layer))
            tf.compat.v1.summary.scalar('input_max', tf.reduce_max(input_tensor=input_layer))
            tf.compat.v1.summary.scalar('input_min', tf.reduce_min(input_tensor=input_layer))
            
        is_training = (mode==tf.estimator.ModeKeys.TRAIN)
        cur_layer = tf.compat.v1.layers.batch_normalization(input_layer, training=is_training)
        for i, layer_width in enumerate(int(d) for d in hparams['hidden_layer_dims']):
            cur_layer = tf.compat.v1.layers.dense(cur_layer, units=layer_width)
            cur_layer = tf.compat.v1.layers.batch_normalization(cur_layer, training=is_training)
            cur_layer = tf.nn.relu(cur_layer)
            tf.compat.v1.summary.scalar('fully_connected_{}_sparsity'.format(i), tf.nn.zero_fraction(cur_layer))
        cur_layer = tf.compat.v1.layers.dropout(cur_layer, rate=hparams['dropout_rate'], training=is_training)
        logits = tf.compat.v1.layers.dense(cur_layer, units=group_size)
        return logits
    
    return _score_fn

In [13]:
def get_eval_metric_fn():
    """return a dict from name to metric functions"""
    metric_fns = {}
#     metric_fns.update({
#         'metric/%s' % name:tfr.metrics.make_ranking_metric_fn(name) for name in [
#             tfr.metrics.RankingMetricKey.ARP, 
#             tfr.metrics.RankingMetricKey.ORDERED_PAIR_ACCURACY
#         ]
#     })
    metric_fns.update({
        'metric/ndcg%d' % topn:tfr.metrics.make_ranking_metric_fn(
            tfr.metrics.RankingMetricKey.NDCG, topn=topn)
        for topn in [10, 20, 100]
    })
    return metric_fns

In [14]:
def train_and_eval(features_train, labels_train, features_vali, labels_vali, hparams):
    """train and evaluate"""
    
    train_input_fn, train_hook = get_train_inputs(features_train, labels_train, batch_size)
    vali_input_fn, vali_hook = get_vali_inputs(features_vali, labels_vali)
    
    optimizer = tf.compat.v1.train.AdagradOptimizer(learning_rate=hparams['learning_rate'])
    
    def _train_op_fn(loss):
        """define train op used in ranking head"""
        update_ops = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.UPDATE_OPS)
        minimize_op = optimizer.minimize(loss=loss, global_step=tf.compat.v1.train.get_global_step())
        train_op = tf.group([minimize_op, update_ops])
        return train_op
    
    ranking_head = tfr.head.create_ranking_head(
        loss_fn=tfr.losses.make_loss_fn(loss), 
        eval_metric_fns=get_eval_metric_fn(), 
        train_op_fn=_train_op_fn)
       
    estimator = tf.estimator.Estimator(
        model_fn=tfr.model.make_groupwise_ranking_fn(
            group_score_fn=make_score_fn(hparams), 
            group_size=group_size, 
            transform_fn=make_transform_fn(), 
            ranking_head=ranking_head), 
#         model_dir=model_dir, 
        config=tf.estimator.RunConfig(save_checkpoints_steps=1000))
    
    train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn, 
        hooks=[train_hook], 
        max_steps=num_train_steps)
    
    vali_spec = tf.estimator.EvalSpec(
        input_fn=vali_input_fn, 
        hooks=[vali_hook], 
        steps=1, 
        exporters=tf.estimator.LatestExporter(
            'latest_exporter', 
            serving_input_receiver_fn=make_serving_input_fn()),
        start_delay_secs=0, 
        throttle_secs=30)
    

    return (estimator, train_spec, vali_spec)

In [15]:
def tuning_and_cross_validation(file_path, tuning_opts, cv=5):
    
    df = pd.read_csv(file_path, sep='\t')

    # hyperparamers tuning
    tuning_results = {}
    for hidden_layer_dims in tuning_opts['hidden_layer_dims']:
        for learning_rate in tuning_opts['learning_rate']:
            for dropout_rate in tuning_opts['dropout_rate']:
                hparams = {
                    'hidden_layer_dims':hidden_layer_dims,
                    'learning_rate':learning_rate,
                    'dropout_rate':dropout_rate
                }
                
                # kfold cross validation
                kf = KFold(n_splits=cv, random_state=23, shuffle=True)
                ndcg100 = []

                for train_idx, vali_idx in kf.split(df):

                    train = df.loc[train_idx]
                    vali = df.loc[vali_idx]

                    features_train, labels_train = dataframe_transformer_train(train)
                    features_vali, labels_vali = dataframe_transformer_train(vali)

                    estimator, train_spec, vali_spec = train_and_eval(features_train, 
                                                                      labels_train, 
                                                                      features_vali, 
                                                                      labels_vali, 
                                                                      hparams)
                    ndcg, path = tf.estimator.train_and_evaluate(estimator, train_spec, vali_spec)
                    ndcg100.append(ndcg['metric/ndcg100'])

                tuning_results.update({
                    str(hparams['hidden_layer_dims']) + ':' +
                    str(hparams['learning_rate']) + ":" +
                    str(hparams['dropout_rate']):
                    np.array(ndcg100).mean()})

    return tuning_results, estimator

    
    
# estimator, train_spec, vali_spec = train_and_eval()
#train and validate
# accuracy = tf.estimator.train_and_evaluate(estimator, train_spec, vali_spec)

In [16]:
def best_hparams(tuning_results):
    for key, value in tuning_results.items():
        if value == max(tuning_results.values()):
            print(key)

In [17]:
if __name__=='__main__':
    ## define constants for later use
    feature_names = [
        'BodyTerms', 'AnchorTerms', 'TitleTerms', 'URLTerms', 'TermsWholeDocument', 'IDFBody', 'IDFAnchor', 'IDFTitle', 
        'IDFURL', 'IDFWholeDocument', 'TFIDFBody', 'TFIDFAnchor', 'TFIDFTitle', 'TFIDFURL', 'TFIDFWholeDocument', 'LengthBody', 
        'LengthAnchor', 'LengthTitle', 'LengthURL', 'LengthWholeDocument', 'BM25Body', 'BM25Anchor', 'BM25Title', 'BM25URL', 
        'BM25WholeDocument', 'LMIRABSBody', 'LMIRABSAnchor', 'LMIRABSTitle', 'LMIRABSURL', 'LMIRABSWholeDocument', 
        'LMIRDIRBody', 'LMIRDIRAnchor', 'LMIRDIRTitle', 'LMIRDIRURL', 'LMIRDIRWholeDocument', 'LMIRIMBody', 'LMIRIMAnchor', 
        'LMIRIMTitle', 'LMIRIMURL', 'LMIRIMWholeDocument', 'PageRank', 'InlinkNum', 'OutlinkNum', 'NumSlashURL', 'LenURL', 
        'NumChildPages'
    ]
    label_name = 'Label'
    list_size = 147 # maximum number of documents correspond to a single query
    batch_size = 32
    num_train_steps = 3000
    group_size = 1
    loss = 'approx_ndcg_loss' 
    train_path = 'train.tsv'
    test_path = 'test.tsv'
    tuning_opts = {
        'hidden_layer_dims':[['64', '32', '16'], ['256', '128']], 
        'learning_rate':[0.01, 0.05],
        'dropout_rate':[0.4, 0.8] 
    }
    
    
    tuning_results, estimator = tuning_and_cross_validation(train_path, tuning_opts, cv=5)
    best_hparams(tuning_results)
    best = {'hidden_layer_dims':['256', '128'], 'learning_rate':0.05, 'dropout_rate':0.8}
    
    df_train = pd.read_csv(train_path, sep='\t')
    train = df_train.head(60000)
    vali = df_train.tail(14216)
    features_train, labels_train = dataframe_transformer_train(train)
    features_vali, labels_vali = dataframe_transformer_train(vali)

    estimator, train_spec, vali_spec = train_and_eval(features_train, labels_train, features_vali, labels_vali, best)
    tf.estimator.train_and_evaluate(estimator, train_spec, vali_spec)
    
    df_test = pd.read_csv(test_path, sep='\t')
    features_pred, qid_to_docs = dataframe_transformer_pred(df_test)
    predict_input_fn, pred_hook = get_predict_inputs(features_pred)
    pred = estimator.predict(input_fn=predict_input_fn, hooks=[pred_hook])
    pred_list = list(pred)
    qid_docid_score_list = [['\t'.join([qid, docid, str(score)]) 
                             for docid, score in zip(qid_to_docs[qid], pred_list[idx])]
                            for idx, qid in enumerate(qid_to_docs.keys())]
    qid_docid_score_list = ['\n'.join(sub_list) for sub_list in qid_docid_score_list]
    qid_docid_score_list = '\n'.join(qid_docid_score_list)
    with open('A2.tsv', 'w') as f:
        f.write(qid_docid_score_list)

INFO:tensorflow:Building groupwise ranking model.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tbfam\\AppData\\Local\\Temp\\tmpwmf6n196', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distrib

C:\Users\tbfam\anaconda3\envs\S3888611\lib\site-packages\keras\legacy_tf_layers\core.py:513: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
C:\Users\tbfam\anaconda3\envs\S3888611\lib\site-packages\keras\engine\base_layer_v1.py:1676: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\tbfam\anaconda3\envs\S3888611\lib\site-packages\keras\legacy_tf_layers\normalization.py:423: UserWarning: `tf.layers.batch_normalization` is deprecated and will be removed in a future version. Please use `tf.keras.layers.BatchNormalization` instead. In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
  warnings.warn(
C:\Users\tbfam\anac

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\tbfam\AppData\Local\Temp\tmpwmf6n196\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = -0.56547755, step = 0
INFO:tensorflow:global_step/sec: 57.7365
INFO:tensorflow:loss = -0.73549455, step = 100 (1.733 sec)
INFO:tensorflow:global_step/sec: 75.838
INFO:tensorflow:loss = -0.66842014, step = 200 (1.319 sec)
INFO:tensorflow:global_step/sec: 74.6525
INFO:tensorflow:loss = -0.66515124, step = 300 (1.340 sec)
INFO:tensorflow:global_step/sec: 74.9884
INFO:tensorflow:loss = -0.72942877, step 

INFO:tensorflow:Loss for final step: -0.7218464.
INFO:tensorflow:Building groupwise ranking model.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tbfam\\AppData\\Local\\Temp\\tmpp75zsn13', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worke

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3000: C:\Users\tbfam\AppData\Local\Temp\tmpp75zsn13\model.ckpt-3000
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: ['serving_default', 'regression']
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from C:\Users\tbfam\AppData\Local\Temp\tmpp75zsn13\model.ckpt-3000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: C:\Users\tbfam\AppData\Local\Temp\tmpp75zsn13\export\latest_exporter\temp-1633838212\saved_model.pb
INFO:tensorflow:Loss for final step: -0.6944846.
INFO:tensorflow:Building groupwise ranking model.
INFO:tensorflow:Using con

...

INFO:tensorflow:global_step/sec: 11.0459
INFO:tensorflow:loss = -0.67755955, step = 100 (9.055 sec)
INFO:tensorflow:global_step/sec: 11.4811
INFO:tensorflow:loss = -0.71980864, step = 200 (8.710 sec)
INFO:tensorflow:global_step/sec: 11.5167
INFO:tensorflow:loss = -0.71296984, step = 300 (8.683 sec)
INFO:tensorflow:global_step/sec: 11.5247
INFO:tensorflow:loss = -0.74781275, step = 400 (8.677 sec)
INFO:tensorflow:global_step/sec: 11.4969
INFO:tensorflow:loss = -0.78796715, step = 500 (8.698 sec)
INFO:tensorflow:global_step/sec: 11.407
INFO:tensorflow:loss = -0.699954, step = 600 (8.767 sec)
INFO:tensorflow:global_step/sec: 9.33337
INFO:tensorflow:loss = -0.6906586, step = 700 (10.715 sec)
INFO:tensorflow:global_step/sec: 11.3014
INFO:tensorflow:loss = -0.7124657, step = 800 (8.846 sec)
INFO:tensorflow:global_step/sec: 12.7069
INFO:tensorflow:loss = -0.67565167, step = 900 (7.870 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 1000...
INFO:tensorflow:Saving che

INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: C:\Users\tbfam\AppData\Local\Temp\tmptf6qhyjc\export\latest_exporter\temp-1633844390\saved_model.pb
INFO:tensorflow:Loss for final step: -0.66033155.
INFO:tensorflow:Building groupwise ranking model.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\tbfam\\AppData\\Local\\Temp\\tmp1oa3ad2o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True

INFO:tensorflow:global_step/sec: 11.0976
INFO:tensorflow:loss = -0.75077105, step = 2200 (9.010 sec)
INFO:tensorflow:global_step/sec: 11.1531
INFO:tensorflow:loss = -0.7103981, step = 2300 (8.966 sec)
INFO:tensorflow:global_step/sec: 11.1419
INFO:tensorflow:loss = -0.7527789, step = 2400 (8.975 sec)
INFO:tensorflow:global_step/sec: 11.0768
INFO:tensorflow:loss = -0.6884445, step = 2500 (9.029 sec)
INFO:tensorflow:global_step/sec: 11.1718
INFO:tensorflow:loss = -0.5682224, step = 2600 (8.964 sec)
INFO:tensorflow:global_step/sec: 11.1086
INFO:tensorflow:loss = -0.7959235, step = 2700 (8.988 sec)
INFO:tensorflow:global_step/sec: 11.1718
INFO:tensorflow:loss = -0.73161256, step = 2800 (8.952 sec)
INFO:tensorflow:global_step/sec: 11.1606
INFO:tensorflow:loss = -0.78569615, step = 2900 (8.960 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 3000...
INFO:tensorflow:Saving checkpoints for 3000 into C:\Users\tbfam\AppData\Local\Temp\tmp1oa3ad2o\model.ckpt.
INFO:tensorf

INFO:tensorflow:loss = -0.7012875, step = 1600 (9.163 sec)
INFO:tensorflow:global_step/sec: 6.86841
INFO:tensorflow:loss = -0.7184933, step = 1700 (14.559 sec)
INFO:tensorflow:global_step/sec: 15.2901
INFO:tensorflow:loss = -0.64547706, step = 1800 (6.540 sec)
INFO:tensorflow:global_step/sec: 11.1222
INFO:tensorflow:loss = -0.7570025, step = 1900 (8.992 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 2000...
INFO:tensorflow:Saving checkpoints for 2000 into C:\Users\tbfam\AppData\Local\Temp\tmps5re4gut\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 2000...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-10T16:48:25
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\tbfam\AppData\Local\Temp\tmps5re4gut\model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evalu

INFO:tensorflow:Saving checkpoints for 1000 into C:\Users\tbfam\AppData\Local\Temp\tmp4lzgcyq4\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 1000...
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-10-10T16:51:58
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\tbfam\AppData\Local\Temp\tmp4lzgcyq4\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Inference Time : 0.71523s
INFO:tensorflow:Finished evaluation at 2021-10-10-16:51:59
INFO:tensorflow:Saving dict for global step 1000: global_step = 1000, labels_mean = 0.25766742, logits_mean = -0.48142081, loss = -0.7523934, metric/ndcg10 = 0.5170006, metric/ndcg100 = 0.5492498, metric/ndcg20 = 0.5353606
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1000: C:\Users\tbfam\AppData\Local\Tem